In [2]:
import pandas as pd
import openpyxl
from tqdm import tqdm
import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import warnings
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import math
from operator import itemgetter

import unidecode
import string
import re
from fuzzywuzzy import fuzz
import os

import glob as f

## Dependientes

In [3]:
print("SGP_educaciontot SGP_educacion_ps SGP_educacion_c SGP_educacion_cg SGP_educacion_cm SGP_saludtot inv_en_salud DI_capadmin SGP_pinfancia IGA_atencion_ciud IGA_sisben".split(" "))

['SGP_educaciontot', 'SGP_educacion_ps', 'SGP_educacion_c', 'SGP_educacion_cg', 'SGP_educacion_cm', 'SGP_saludtot', 'inv_en_salud', 'DI_capadmin', 'SGP_pinfancia', 'IGA_atencion_ciud', 'IGA_sisben']


### Panel municipal

In [4]:
bg = pd.read_stata("CEDE//PANEL_DE_EDUCACION(2021).dta")
bg = bg[['codmpio', 'ano', 'docen_total', 'n_icfes', 'pobsimple_primaria', 'pobsimple_secund', 'ind_alfa']]
bg = bg.rename(columns= {"ano": "date"})
bg['date'] = bg['date'].dt.year
ed = bg.copy()

In [5]:
bg = pd.read_stata("CEDE//PANEL_CARACTERISTICAS_GENERALES(2022).dta")
bg = bg[['codmpio', 'ano', 'gandina', 'gcaribe', 'gpacifica', 'gorinoquia', 'gamazonia', 'nbi', 'pobreza', 'gini', 'altura', 'areaoficialkm2']]
bg = bg.rename(columns= {"ano": "date"})
# bg['date'] = bg['date'].dt.year
cg = bg.copy()

In [9]:
bg = pd.read_stata("CEDE//PANEL_BUEN_GOBIERNO(2022).dta")
bg = bg[['codmpio', 'ano', 'SGP_educaciontot', 'SGP_educacion_ps', 'SGP_educacion_c', 'SGP_educacion_cg', 'SGP_educacion_cm', 'SGP_saludtot', 'inv_en_salud', 'DI_capadmin', 'SGP_pinfancia', 'IGA_atencion_ciud', 'IGA_sisben']]
bg = bg.rename(columns= {"ano": "date"})
# bg['date'] = bg['date'].dt.year
bg = bg.copy()

In [10]:
df  = pd.merge(ed,cg, "outer", ["codmpio", "date"])
df  = pd.merge(df,bg, "outer", ["codmpio", "date"])

In [ ]:
df.to_csv("controles.csv", index = False)